In [1]:
import re
import os
from multiprocessing import Pool
from glom import glom

import numpy as np
import pandas as pd


import yaml
import json
import xmltodict

import requests
from urllib.request import urlretrieve, urlopen

In [2]:
folder = '../data/'
def forceDir(dirName):
    if not os.path.exists(dirName):
        os.makedirs(dirName)
    return dirName


index_folder = forceDir('us990/index/')
bmf_folder = forceDir('bmf/')
whitelist_folder = forceDir('whitelist/')
lists_folder = forceDir('lists/')
f990_folder = forceDir('../data/us990/')
json_folder = forceDir('us990/json/')


In [4]:
z = urlopen(download_uri % 1)

In [5]:
pd.read_csv(z)

,EIN,NAME,ICO,STREET,CITY,STATE,ZIP,GROUP,SUBSECTION,AFFILIATION,...,ASSET_CD,INCOME_CD,FILING_REQ_CD,PF_FILING_REQ_CD,ACCT_PD,ASSET_AMT,INCOME_AMT,REVENUE_AMT,NTEE_CD,SORT_NAME
0,19818,PALMER SECOND BAPTIST CHURCH,NaN,1050 THORNDIKE ST,PALMER,MA,01069-1507,3125,3,9,...,0,0,6,0,12,NaN,NaN,NaN,NaN,3514
1,29215,ST GEORGE CATHEDRAL,NaN,523 E BROADWAY,SOUTH BOSTON,MA,02127-4415,2365,3,9,...,0,0,6,0,12,NaN,NaN,NaN,NaN,NaN
2,587764,IGLESIA BETHESDA INC,NaN,157 ANDOVER ST,LOWELL,MA,01852-2348,0,3,3,...,0,0,6,0,12,NaN,NaN,NaN,X21,NaN
3,635913,MINISTERIO APOSTOLICO JESUCRISTO ES EL SENOR INC,NaN,454 ESSEX ST,LAWRENCE,MA,01840-1242,0,3,3,...,0,0,6,0,12,NaN,NaN,NaN,X21,NaN
4,765634,MERCY CHAPEL INTERNATIONAL,NaN,75 MORTON VILLAGE DR APT 408,MATTAPAN,MA,02126-2433,0,3,3,...,0,0,6,0,12,NaN,NaN,NaN,X20,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251620,990383605,AMICI ITALIAN AMERICNA BENEVOLENT ASSOCIATION INC,% LOUIS MESSERE,PO BOX 7413,SHREWSBURY,NJ,07702-7413,0,3,3,...,0,0,2,0,12,0.0,0.0,0.0,T12,NaN
251621,990384215,ANGKOR HOSPITAL FOR CHILDREN INC,% CORPORATION TRUST COMPANY,139 PLOWDEN RD,PUTNEY,VT,05346-8694,0,3,3,...,3,4,1,0,12,53025.0,384280.0,384280.0,T30,NaN
251622,990384539,HONOR SOCIETY OF PHI KAPPA PHI,% EILEEN M BEITCR,4245 EAST AVE,ROCHESTER,NY,14618-3703,3488,3,9,...,0,0,2,0,6,0.0,0.0,0.0,NaN,320 NAZARETH COLLEGE HONOR SOCIETY
251623,990536495,INTERNATIONAL ASSOCIATION OF CHINESE LINGUISTICS,% FENG-HSI LIU,4400 VESTAL PKWY,BINGHAMTON,NY,13902-4600,0,3,3,...,0,0,2,0,12,0.0,0.0,0.0,B99,NaN


In [3]:
# download all regions
download_uri = 'https://www.irs.gov/pub/irs-soi/eo%d.csv'

In [ ]:
regiondfs = []
for i in range(1,5):
    buf = urlopen(download_uri % i)    
    regiondfs.append(pd.read_csv(buf))
    

In [9]:
bmf = pd.concat(regiondfs, ignore_index=True)
bmf.EIN = bmf.EIN.astype(str).str.pad(9, fillchar='0')
bmf.ACTIVITY = bmf.ACTIVITY.astype(str).str.pad(9, fillchar='0')
print(bmf.shape)
print(bmf.dtypes)
bmf.head(2)

(1716242, 28)
EIN                  object
NAME                 object
ICO                  object
STREET               object
CITY                 object
STATE                object
ZIP                  object
GROUP                 int64
SUBSECTION            int64
AFFILIATION           int64
CLASSIFICATION        int64
RULING                int64
DEDUCTIBILITY         int64
FOUNDATION            int64
ACTIVITY             object
ORGANIZATION          int64
STATUS                int64
TAX_PERIOD          float64
ASSET_CD              int64
INCOME_CD             int64
FILING_REQ_CD         int64
PF_FILING_REQ_CD      int64
ACCT_PD               int64
ASSET_AMT           float64
INCOME_AMT          float64
REVENUE_AMT         float64
NTEE_CD              object
SORT_NAME            object
dtype: object


,EIN,NAME,ICO,STREET,CITY,STATE,ZIP,GROUP,SUBSECTION,AFFILIATION,...,ASSET_CD,INCOME_CD,FILING_REQ_CD,PF_FILING_REQ_CD,ACCT_PD,ASSET_AMT,INCOME_AMT,REVENUE_AMT,NTEE_CD,SORT_NAME
0,000019818,PALMER SECOND BAPTIST CHURCH,NaN,1050 THORNDIKE ST,PALMER,MA,01069-1507,3125,3,9,...,0,0,6,0,12,NaN,NaN,NaN,NaN,3514
1,000029215,ST GEORGE CATHEDRAL,NaN,523 E BROADWAY,SOUTH BOSTON,MA,02127-4415,2365,3,9,...,0,0,6,0,12,NaN,NaN,NaN,NaN,NaN


In [24]:
bmf = bmf.set_index('EIN')
bmf.to_csv(bmf_folder + 'raw_usa.csv')
bmf.to_pickle(bmf_folder + 'raw_usa.pkl')

## Activity

In [17]:
activity = bmf.ACTIVITY.map(lambda x: (x[0:3],x[3:-3],x[-3:]))

In [19]:
# elongate activity
activity = activity.explode().\
            reset_index().drop_duplicates().\
            set_index('EIN')

In [20]:
# drop zeroes
activity = activity[activity.ACTIVITY != '000']

In [22]:
print(activity.shape)
activity.head(2)

(1356007, 1)


,ACTIVITY
EIN,
000019818,001
000029215,001


In [23]:
activity.to_csv(bmf_folder + 'raw_activities.csv')
activity.to_pickle(bmf_folder + 'raw_activities.pkl')

### Filter by whitelist

In [27]:
whitelist_activity = '001,031,032,061,126,150,152,153,154,155,156,158,160,165,166,179,261,296,297,320,322,323,325,326,327,328,321,324,349,351,380,381,382,383,384,401,406,407,430,431,432,449,460,461,462,463,560,561,562,563,564,565,566,567,568,569,572,573,574,900,902,907,913,915,916,917,990'.split(',')
whitelist_ntee = 'B21,B28,B6,B7,B9,C2,D2,E2,E3,E4,E5,E6,E7,E8,E9,F2,F3,F4,F5,F6,F7,F8,F9,G2,G3,G4,G5,G6,G7,G8,G9,I2,I5,I6,I7,I8,J2,J3,K3,K4,K5,L2,L3,L4,L8,M2,N2,N3,O2,O3,P2,P3,P4,P5,P6,P7,P8,P99,R2,R3,R6,W3,W4,W5,W8,W9,X2,X3,X4,X5,X7,X9'.split(',')

#### by activity codes

In [28]:
has_good_activity = activity.isin(whitelist_activity).ACTIVITY
uniq_indices = activity[has_good_activity].index.drop_duplicates()

good_activity = bmf.loc[uniq_indices]
print(good_activity.shape)
good_activity.head(2)

(318266, 27)


,NAME,ICO,STREET,CITY,STATE,ZIP,GROUP,SUBSECTION,AFFILIATION,CLASSIFICATION,...,ASSET_CD,INCOME_CD,FILING_REQ_CD,PF_FILING_REQ_CD,ACCT_PD,ASSET_AMT,INCOME_AMT,REVENUE_AMT,NTEE_CD,SORT_NAME
EIN,,,,,,,,,,,,,,,,,,,,,
000019818,PALMER SECOND BAPTIST CHURCH,NaN,1050 THORNDIKE ST,PALMER,MA,01069-1507,3125,3,9,7000,...,0,0,6,0,12,NaN,NaN,NaN,NaN,3514
000029215,ST GEORGE CATHEDRAL,NaN,523 E BROADWAY,SOUTH BOSTON,MA,02127-4415,2365,3,9,7000,...,0,0,6,0,12,NaN,NaN,NaN,NaN,NaN


#### by ntee codes

In [29]:
search_rx = '^(%s)' % '|'.join(whitelist_ntee)
good_ntee = bmf[bmf['NTEE_CD'].str.contains(search_rx, na=False)]

/usr/lib/python3.7/site-packages/pandas/core/strings.py:1843: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [30]:
union = pd.concat([good_activity, good_ntee]).drop_duplicates()
print(union.shape)
union.head(2)

(726551, 27)


,NAME,ICO,STREET,CITY,STATE,ZIP,GROUP,SUBSECTION,AFFILIATION,CLASSIFICATION,...,ASSET_CD,INCOME_CD,FILING_REQ_CD,PF_FILING_REQ_CD,ACCT_PD,ASSET_AMT,INCOME_AMT,REVENUE_AMT,NTEE_CD,SORT_NAME
EIN,,,,,,,,,,,,,,,,,,,,,
000019818,PALMER SECOND BAPTIST CHURCH,NaN,1050 THORNDIKE ST,PALMER,MA,01069-1507,3125,3,9,7000,...,0,0,6,0,12,NaN,NaN,NaN,NaN,3514
000029215,ST GEORGE CATHEDRAL,NaN,523 E BROADWAY,SOUTH BOSTON,MA,02127-4415,2365,3,9,7000,...,0,0,6,0,12,NaN,NaN,NaN,NaN,NaN


In [31]:
union.to_csv(bmf_folder + 'usa.csv')
union.to_pickle(bmf_folder + 'usa.pkl')

### Filter down to just Missouri

In [32]:
missouri = union[union.STATE == 'MO']
missouri.to_csv(bmf_folder + 'missouri_bmf.csv')
missouri.to_pickle(bmf_folder + 'missouri_bmf.pkl')

print(missouri.shape)
missouri.head(3)

(16122, 27)


,NAME,ICO,STREET,CITY,STATE,ZIP,GROUP,SUBSECTION,AFFILIATION,CLASSIFICATION,...,ASSET_CD,INCOME_CD,FILING_REQ_CD,PF_FILING_REQ_CD,ACCT_PD,ASSET_AMT,INCOME_AMT,REVENUE_AMT,NTEE_CD,SORT_NAME
EIN,,,,,,,,,,,,,,,,,,,,,
000360268,IGLESIA VICTORIA,NaN,1445 N BOONVILLE AVE,SPRINGFIELD,MO,65802-1894,1678,3,9,7000,...,0,0,6,0,12,NaN,NaN,NaN,NaN,NaN
010605038,HABITAT FOR HUMANITY INTERNATIONAL INC,% EXECUTIVE DIRECTOR,PO BOX 223,CASSVILLE,MO,65625-0223,8545,3,9,1000,...,4,4,1,0,6,413721.0,116609.0,116609.0,L20Z,HFH OF SOUTH BARRY CO MO
010829991,MORLEY CHURCH OF GOD,% JEANNIE E WILLIAMS,PO BOX 267,MORLEY,MO,63767-0267,1489,3,9,7000,...,0,0,6,0,12,NaN,NaN,NaN,NaN,NaN
